In [1]:
import ROOT as rt
import csv
import re
import sys
import collections
from collections import OrderedDict
import uproot
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
from numpy import linalg as LA
from ROOT import TLorentzVector
from array import array
# import PyTEX
import os

import math
import random
from optparse import OptionParser

sys.path.append('/storage/user/jmao/gpu/jmao/cms-llp/delayed_jet_analyzer/lib/')

from histo_utilities import create_TH1D, create_TH2D, create_TGraph,std_color_list, make_ratio_plot

import CMS_lumi, tdrstyle 
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Simulation Preliminary"

     
print(sys.version)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
import pandas as pd
import pickle as pickle
import shlex
import uproot
import seaborn as sns

from scipy.spatial.distance import cdist
from collections import Counter 
import datetime
import pytz

from prettytable import PrettyTable
from pyjet import cluster

from itertools import chain 

import xgboost as xgb
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier


Welcome to JupyROOT 6.22/02
3.6.8 (default, Apr  2 2020, 13:34:55) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


In [2]:
## Inputs
#labels
tags = []
tags = [
 
        'H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017',
        'H(bb)H(bb), N3N2, 300 GeV, 1 GeV, 1 m, 2017',
        'H(bb)H(bb), N3N2, 250 GeV, 1 GeV, 1 m, 2017',
    
        'H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2016',
        'H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2017',
        'H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2018',
    
        'H(bb)H(bb), N3N2, 175 GeV, 1 GeV, 1 m, 2017',
        'H(bb)H(bb), N3N2, 150 GeV, 1 GeV, 1 m, 2017',
        'H(bb)H(bb), N3N2, 127 GeV, 1 GeV, 1 m, 2017',
        ]


# directory
home_dir = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetTimingAnalyzer/V1p17/v21/'

# file names

fnames = {}

fnames['H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2016'] =  'MC_Summer16/normalized/n3n2-n1-hbb-hbb_mh200_pl1000_ev100000_1pb_weighted.root'
fnames['H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2017'] =  'MC_Fall17/normalized/n3n2-n1-hbb-hbb_mh200_pl1000_ev100000_1pb_weighted.root'
fnames['H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2018'] =  'MC_Fall18/normalized/n3n2-n1-hbb-hbb_mh200_pl1000_ev100000_1pb_weighted.root'


fnames['H(bb)H(bb), N3N2, 127 GeV, 1 GeV, 1 m, 2017'] =  'MC_Fall17/normalized/n3n2-n1-hbb-hbb_mh127_pl1000_ev100000_1pb_weighted.root'
fnames['H(bb)H(bb), N3N2, 150 GeV, 1 GeV, 1 m, 2017'] =  'MC_Fall17/normalized/n3n2-n1-hbb-hbb_mh150_pl1000_ev100000_1pb_weighted.root'
fnames['H(bb)H(bb), N3N2, 175 GeV, 1 GeV, 1 m, 2017'] =  'MC_Fall17/normalized/n3n2-n1-hbb-hbb_mh175_pl1000_ev100000_1pb_weighted.root'
fnames['H(bb)H(bb), N3N2, 250 GeV, 1 GeV, 1 m, 2017'] =  'MC_Fall17/normalized/n3n2-n1-hbb-hbb_mh250_pl1000_ev100000_1pb_weighted.root'
fnames['H(bb)H(bb), N3N2, 300 GeV, 1 GeV, 1 m, 2017'] =  'MC_Fall17/normalized/n3n2-n1-hbb-hbb_mh300_pl1000_ev100000_1pb_weighted.root'
fnames['H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017'] =  'MC_Fall17/normalized/n3n2-n1-hbb-hbb_mh400_pl1000_ev100000_1pb_weighted.root'
 

# get all the trees
tree = OrderedDict()

for tag in tags:
    print(home_dir+fnames[tag])

    root_dir = uproot.open(home_dir+fnames[tag])
    tree[tag] = root_dir['SusyLLPTree']

    print ('Open ready')
    print(tag)
    print ('Tree ready')
    print(tree[tag], tree)
    v = tree[tag]
    jet_amax = v['jetGammaMax_ET'].array()
    print(jet_amax[:10])
    jet_amax = v['weight'].array()
    print(jet_amax[:10])
    if 'back' not in tag:
        calo_jet_amax = v['gLLP0_EB'].array()
        print(calo_jet_amax[:10])
        calo_jet_amax = v['jetNRecHitsHcal'].array()
        print(calo_jet_amax[:10])
        calo_jet_amax = v['jetTimeRecHitsHcal'].array()
        print(calo_jet_amax[:10])

#     nhcalhit = v['jetNRecHitsHcal'].array()
#     hcalt = v['jetTimeRecHitsHcal'].array()

/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetTimingAnalyzer/V1p17/v21/MC_Fall17/normalized/n3n2-n1-hbb-hbb_mh400_pl1000_ev100000_1pb_weighted.root
Open ready
H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017
Tree ready
<TTree b'SusyLLPTree' at 0x7f84a6c3ba20> OrderedDict([('H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017', <TTree b'SusyLLPTree' at 0x7f84a6c3ba20>)])
[[0.021735515 0.022792224 0.12800717] [0.07487983 0.0 0.0] [0.010533664 0.009684265 0.037519798] ... [0.0025499675 0.0] [0.010867204] [0.024418194 0.13759252 0.0 0.34890115 0.61975765 0.047106914]]
[6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07
 6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07]
[ True False  True  True  True False False  True  True False]
[[15 0 0] [25 6 0] [28 0 9] ... [18 3] [6] [0 0 7 0 0 0]]
[[2.7225251 -100.0 -100.0] [1.6583755 4.4335413 -100.0] [1.8135751 -100.0 0.449813] ... [2.2599294 -1.3800132] [1.8571517] [-100.0 -100.0 4.7095222 -100.0 -100.0 -100.0]]
/mn

In [3]:
## Variables
trg_path = 467
#HLT_PFMETNoMu120_PFMHTNoMu120_IDTight

# met_cut = 150
met_cut = 200
jet_pt_cut = 30
jet_eta_cut = 1.48
njet_cut = 1
nlep_cut = 0


trg_jet_pt = {}
trg_jet_eta = {}
trg_jet_phi = {}
trg_jet_e = {}
trg_jet_met = {}
trg_jet_ht = {}
trg_jet_w = {}
trg_flat_w = {}


trg_jet_hcalt = {}
trg_jet_nhcalhit = {}

trg_jet_t = {}
trg_jet_tmf = {}
trg_jet_chef = {}
trg_jet_dr = {}

trg_jet_nj = {}

 
nTags = {}

for k,v in tree.items():
    print(k,v)


    trg_jet_pt[k] = {}
    trg_jet_met[k] = {}
    trg_jet_ht[k] = {}
    trg_jet_w[k] = {}
    trg_flat_w[k] = {}

    # branches
    hlt = v['HLTDecision'].array()[:,trg_path]
#     hlt = v['HLTDecision'].array()
   
    met = v['met'].array()

    nl = v['nLeptons'].array()
    nj = v['nJets'].array()
    pt = v['jetPt'].array()
    eta = v['jetEta'].array()
    phi = v['jetPhi'].array()
    e = v['jetE'].array()
    
    nhcalhit = v['jetNRecHitsHcal'].array()
    hcalt = v['jetTimeRecHitsHcal'].array()
    
    
    t = v['jetTime'].array()
    tmf = v['jetGammaMax_ET'].array()
    chef = v['jetChargedHadronEnergyFraction'].array()
    dr = v['jetMinDeltaRPVTracks'].array()
    
 
    
    weight = v['weight'].array()
    
    print(weight.shape)
    print(pt.shape)
    
    sel_hlt_triggered = 1
    sel_hlt_str  = np.logical_and(sel_hlt_triggered, hlt)
#     sel_hlt_str  = np.logical_and(sel_hlt_triggered, hlt[:,trg_path])
    
    basic_met_str = np.logical_and(nj>=njet_cut, met>met_cut)   #before/after trigger case
    basic_met_str = np.logical_and(basic_met_str, nl==nlep_cut)
#     basic_met_str = np.logical_and(nj>=0, met>0)   # before/after pre-selection case
    
    basic_jet_str = np.logical_and(abs(eta)<jet_eta_cut, pt>jet_pt_cut)   
#     basic_jet_str = np.logical_and(basic_jet_str, nhcalhit>0)   
#     basic_jet_str = np.logical_and(basic_jet_str, hcalt>-100)   

    # matched requirement
    if 'background' not in k:
        matched0 = v['jet_matched_gLLP0_grandaughter'].array()
        matched1 = v['jet_matched_gLLP1_grandaughter'].array()

        matched_str = np.logical_or(matched0==1, matched1==1)
#         matched_str = np.logical_or(1, 1)
        
#         llp_mdr = v['gLLP_grandaughter_min_delta_r_match_jet'].array()

        acc0_eb = v['gLLP0_EB'].array()
        acc1_eb = v['gLLP1_EB'].array()

        acc_eb_str = np.logical_or(acc0_eb==1, acc1_eb==1)
#         acc_eb_str = np.logical_or(1, 1)

#         basic_met_str = np.logical_and(basic_met_str, acc_eb_str)

#         basic_jet_str = np.logical_and(basic_jet_str, matched_str)
#         llp_mdr_str = np.logical_and(llp_mdr[:,0]<0.4, llp_mdr[:,1]<0.4)
#         llp_mdr_str = np.logical_and(llp_mdr_str, llp_mdr[:,2]<0.4)
#         llp_mdr_str = np.logical_and(llp_mdr_str, llp_mdr[:,3]<0.4)
#         basic_jet_str = np.logical_and(basic_jet_str, llp_mdr_str)
        
#     basic_trg_str = np.logical_and(basic_str, sel_hlt_str)
    met_trg_str = np.logical_and(basic_met_str, sel_hlt_str)

#     #TAGS
#     tagged_jet = np.logical_and(tmf<tmf_et_cut, dr>delta_r_cut)
#     tagged_jet = np.logical_and(tagged_jet, t>time_cut)
#     tagged_jet = np.logical_and(tagged_jet, chef<chef_cut)

#     nTags[k] = pt[tagged_jet].count() #event level variable, number of tags jet per event, no trg_jet_ws added

#     #EVNET CUT STRING
#     met_trg_str = np.logical_and(met_trg_str, nTags[k]>=2)
    
#     basic_jet_str = np.logical_and(basic_jet_str, llp_mdr<0.4)

    #apply basic cut string
    # separate jagged jet array req. and event level req.
    trg_jet_pt[k] = pt[basic_jet_str][met_trg_str]
    trg_jet_eta[k] = eta[basic_jet_str][met_trg_str]
    trg_jet_phi[k] = phi[basic_jet_str][met_trg_str]
    trg_jet_e[k] = e[basic_jet_str][met_trg_str]
    trg_jet_met[k] = met[met_trg_str]
    trg_jet_w[k] = weight[met_trg_str]
    trg_jet_nj[k] = nj[met_trg_str]
    
    trg_jet_nhcalhit[k] = nhcalhit[basic_jet_str][met_trg_str]
    trg_jet_hcalt[k] = hcalt[basic_jet_str][met_trg_str]
    
    trg_jet_t[k] = t[basic_jet_str][met_trg_str]
    trg_jet_tmf[k] = tmf[basic_jet_str][met_trg_str]
    trg_jet_chef[k] = chef[basic_jet_str][met_trg_str]
    trg_jet_dr[k] = dr[basic_jet_str][met_trg_str]
    
 
    
    #remove empty part in jet
    trg_jet_met[k] = trg_jet_met[k][trg_jet_pt[k].count()>0]
    trg_jet_w[k] = trg_jet_w[k][trg_jet_pt[k].count()>0]
    trg_jet_nj[k] = trg_jet_nj[k][trg_jet_pt[k].count()>0]
    
    trg_jet_nhcalhit[k] = trg_jet_nhcalhit[k][trg_jet_pt[k].count()>0]
    trg_jet_hcalt[k] = trg_jet_hcalt[k][trg_jet_pt[k].count()>0]
    
    trg_jet_t[k] = trg_jet_t[k][trg_jet_pt[k].count()>0]
    trg_jet_tmf[k] = trg_jet_tmf[k][trg_jet_pt[k].count()>0]
    trg_jet_chef[k] = trg_jet_chef[k][trg_jet_pt[k].count()>0]
    trg_jet_dr[k] = trg_jet_dr[k][trg_jet_pt[k].count()>0]
    
 
    
    
    trg_jet_eta[k] = trg_jet_eta[k][trg_jet_pt[k].count()>0]
    trg_jet_phi[k] = trg_jet_phi[k][trg_jet_pt[k].count()>0]
    trg_jet_e[k] = trg_jet_e[k][trg_jet_pt[k].count()>0]
    
    trg_jet_pt[k] = trg_jet_pt[k][trg_jet_pt[k].count()>0]
    

    
    
#     #flat weight
    trg_flat_w[k] = [np.array([x]*len(y)) for x,y in zip(trg_jet_w[k],trg_jet_pt[k])]
    trg_flat_w[k] = list(chain.from_iterable(trg_flat_w[k])) 
    
    #print out check
 
    print(hlt[basic_met_str][:10])

#     print(len(weight[sel_hlt_str]))

#     print(trg_jet_pt[k][:10])
#     print(trg_jet_met[k][:10])
    print(trg_jet_w[k][:10])
#     print(trg_jet_nj[k][:10])
#     print(len(trg_jet_pt[k]))
#     print(len(trg_jet_met[k]))
    print(len(trg_jet_w[k]))
    print(trg_jet_pt[k][:10])
    print(trg_jet_hcalt[k][:10])
    print(trg_jet_nhcalhit[k][:10])
#     print(nTags[k][:10])
    print(trg_jet_t[k][:10])
    print(trg_jet_tmf[k][:10])
    print(trg_jet_chef[k][:10])
    print(trg_jet_dr[k][:10])
#     print(sum(trg_jet_w[k]))
#     print(len(trg_jet_w[k]))
#     print(hlt[met_trg_str][:10])
    
#     print(pf_flat_w[k][:10])
#     print(trg_flat_w[k][:10])
    

H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017 <TTree b'SusyLLPTree' at 0x7f84a6c3ba20>
(13315,)
(13315,)
[ True  True  True  True  True  True  True  True  True  True]
[6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07
 6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07]
12452
[[205.91058 50.006046 32.958202] [266.31104 68.43851 50.33762] [304.05515 109.38851 54.389904] ... [444.14822 82.740654] [192.5565] [170.3947 74.125435 67.90879 40.573574 35.692177 34.624924]]
[[2.7225251 -100.0 -100.0] [1.6583755 4.4335413 -100.0] [1.8135751 -100.0 0.449813] ... [2.2599294 -1.3800132] [1.8571517] [-100.0 -100.0 4.7095222 -100.0 -100.0 -100.0]]
[[15 0 0] [25 6 0] [28 0 9] ... [18 3] [6] [0 0 7 0 0 0]]
[[1.2872763 1.9330175 1.7513475] [0.21516937 1.3543732 4.5540953] [0.83097625 1.4978012 1.5215455] ... [0.32414672 0.12591136] [nan] [0.7905247 0.11985159 0.24297702 -1.1924617 -0.632529 1.8384988]]
[[0.021735515 0.022792224 0.12800717] [0.07487983 0.0 0.0] [0.010533664 0.0

In [49]:
time_cuts = [0.08676443430239597] 
tmf_et_cuts = [0.16335152541277656] 
delta_r_cuts = [0.06402562462487048]
chef_cuts = [0.061590940809278155]

 
validity = []
significance = []
nTags = {}
nJets = {}
for chef_cut in chef_cuts:
    for tmf_et_cut in tmf_et_cuts: #loop over the three variables
        for time_cut in time_cuts:
            for delta_r_cut in delta_r_cuts:
                #go through each sample
    #         
                for k,T in trg_jet_tmf.items():
                    print(k, len(T))
                    tagged_jet = np.logical_and(trg_jet_tmf[k]<tmf_et_cut, trg_jet_dr[k]>delta_r_cut)
                    tagged_jet = np.logical_and(tagged_jet, trg_jet_t[k]>time_cut)
                    tagged_jet = np.logical_and(tagged_jet, trg_jet_chef[k]<chef_cut)

                    nTags[k] = trg_jet_pt[k][tagged_jet].count() #event level variable, number of tags jet per event, no trg_jet_ws added
#                     print(k, 'bin 0 ', len(trg_jet_w[k][nTags[k]==0]))
#                     print(k, 'bin 1 ', len(trg_jet_w[k][nTags[k]==1]))
#                     print(k, 'bin 2 ', len(trg_jet_w[k][nTags[k]>=2]))
                    print(k, 'bin 0 ', sum(trg_jet_w[k][nTags[k]==0]))
                    print(k, 'bin 1 ', sum(trg_jet_w[k][nTags[k]==1]))
                    print(k, 'bin 2 ', sum(trg_jet_w[k][nTags[k]>=2]))
                    print('unique nTags element: ', np.unique(nTags[k]))
                    for i,ele in enumerate(nTags[k]):
#                         print('nTags element: ', i, ' th, ', ele)
                        if ele > 2:
                            nTags[k][i] = 2
                

H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017 12452
H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017 bin 0  0.003495055965345273
H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017 bin 1  0.0030799343755916198
H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017 bin 2  0.0010043264268233543
unique nTags element:  [0 1 2 3 4]
H(bb)H(bb), N3N2, 300 GeV, 1 GeV, 1 m, 2017 6823
H(bb)H(bb), N3N2, 300 GeV, 1 GeV, 1 m, 2017 bin 0  0.006539228024848853
H(bb)H(bb), N3N2, 300 GeV, 1 GeV, 1 m, 2017 bin 1  0.004880132457401487
H(bb)H(bb), N3N2, 300 GeV, 1 GeV, 1 m, 2017 bin 2  0.0013283974644764385
unique nTags element:  [0 1 2 3 4]
H(bb)H(bb), N3N2, 250 GeV, 1 GeV, 1 m, 2017 4691
H(bb)H(bb), N3N2, 250 GeV, 1 GeV, 1 m, 2017 bin 0  0.013644162800119375
H(bb)H(bb), N3N2, 250 GeV, 1 GeV, 1 m, 2017 bin 1  0.00869751501340943
H(bb)H(bb), N3N2, 250 GeV, 1 GeV, 1 m, 2017 bin 2  0.0016301452933475957
unique nTags element:  [0 1 2 3]
H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2016 3148
H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2016 bin 0

In [50]:
nJets = {}
nJetsW = {}
nTagsW = {}
for k,v in tree.items():
    print(k,v)
    
    print(np.unique(nTags[k]))
    print(np.unique(trg_jet_w[k]))
    nJets[k] = trg_jet_pt[k].count()
    
    nJetsW[k] = [x*y for x,y in zip(nJets[k], trg_jet_w[k])]
    nTagsW[k] = [x*y for x,y in zip(nTags[k], trg_jet_w[k])]
    
    print(trg_jet_w[k][:5])
    print(trg_jet_pt[k][:5])
    print(nJets[k][:5])
    print(nTags[k][:5])
    print(nJetsW[k][:5])
    print(nTagsW[k][:5])
    
    print(np.sum(nTagsW[k])/np.sum(nJetsW[k]))
    print(np.sum(nTags[k])/np.sum(nJets[k]))
    
    
    

H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017 <TTree b'SusyLLPTree' at 0x7f9e62111b00>
[0 1 2]
[6.086827e-07]
[6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07 6.086827e-07]
[[205.91058 50.006046 32.958202] [266.31104 68.43851 50.33762] [304.05515 109.38851 54.389904] [465.56448 245.60445 185.10541] [258.3931]]
[3 3 3 3 1]
[1 2 1 0 0]
[1.826048048769735e-06, 1.826048048769735e-06, 1.826048048769735e-06, 1.826048048769735e-06, 6.08682682923245e-07]
[6.08682682923245e-07, 1.21736536584649e-06, 6.08682682923245e-07, 0.0, 0.0]
0.251564756860857
0.251564756860857
H(bb)H(bb), N3N2, 300 GeV, 1 GeV, 1 m, 2017 <TTree b'SusyLLPTree' at 0x7f9e61df7860>
[0 1 2]
[1.8683509e-06]
[1.8683509e-06 1.8683509e-06 1.8683509e-06 1.8683509e-06 1.8683509e-06]
[[170.87941 96.84204] [551.3421 296.6329 42.08419] [535.6032 404.2934 70.325615 34.731907] [121.04836] [41.340702]]
[2 3 4 1 1]
[1 0 0 0 1]
[3.736701728485059e-06, 5.6050525927275885e-06, 7.473403456970118e-06, 1.8683508642425295e-06, 1.8683508642425295

In [51]:
from prettytable import PrettyTable
print('\n\n======================= Summary ==============================')
table =  PrettyTable(['Sample', 'Count Num', 'Count Den','Count Ratio', 'Weight Num', 'Weight Den','Weight Ratio'])
for k,v in tree.items():
    idx=6
    row = [k, np.sum(nTags[k]), np.sum(nJets[k]), round (np.sum(nTags[k])/np.sum(nJets[k]), idx) , round(np.sum(nTagsW[k]), idx),  round(np.sum(nJetsW[k]), idx), round (np.sum(nTagsW[k])/np.sum(nJetsW[k]), idx) ,]
    table.add_row(row)
        
print(table)



======================= Summary ==============================
+---------------------------------------------+-----------+-----------+-------------+------------+------------+--------------+
|                    Sample                   | Count Num | Count Den | Count Ratio | Weight Num | Weight Den | Weight Ratio |
+---------------------------------------------+-----------+-----------+-------------+------------+------------+--------------+
| H(bb)H(bb), N3N2, 400 GeV, 1 GeV, 1 m, 2017 |    8360   |   33232   |   0.251565  |  0.005089  |  0.020228  |   0.251565   |
| H(bb)H(bb), N3N2, 300 GeV, 1 GeV, 1 m, 2017 |    4034   |   17544   |   0.229936  |  0.007537  |  0.032778  |   0.229936   |
| H(bb)H(bb), N3N2, 250 GeV, 1 GeV, 1 m, 2017 |    2340   |   11464   |   0.204117  |  0.011958  |  0.058583  |   0.204117   |
| H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2016 |    1310   |    7331   |   0.178693  |  0.011025  |  0.061698  |   0.178693   |
| H(bb)H(bb), N3N2, 200 GeV, 1 GeV, 1 m, 2017 

In [52]:
# plotDir = '/storage/user/jmao/gpu/jmao/cms-llp/plots/20201029/dnn/90ps/acc_match_hcal/'
# plotDir = '/storage/user/jmao/gpu/jmao/cms-llp/plots/20201029/dnn/90ps/acc_match/'
# plotDir = '/storage/user/jmao/gpu/jmao/cms-llp/plots/20201029/dnn/90ps/match/'
plotDir = '/storage/user/jmao/gpu/jmao/cms-llp/plots/20201029/dnn/90ps/no_match/'

os.system("mkdir -p "+ plotDir)

0

In [53]:
table_txt = table.get_string()
with open(plotDir+'/TagEffTable.txt','w') as file:
    file.write(table_txt)